In [ ]:
import cv2
import sklearn
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import math
import glob
import tensorflow as tf
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from matplotlib.image import NonUniformImage
from google.colab.patches import cv2_imshow
import os
from sklearn.svm import SVC


In [ ]:
def get_pixel(img, center, x, y):
    new_value = 0
    try:
        if img[x][y] >= center:
            new_value = 1
    except:
        pass
    return new_value


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
def get_pixel_else_0(l, idx, idy):
    if idx < int(len(l)) - 1 and idy < len(l[0]):
        return l[idx,idy]
    else:
        return 0

In [ ]:
def get_pixel_sign(img, center, x, y):
    new_value = 0
    try:
        if img[x][y] >= center:
            new_value = 1
    except:
        pass
    return new_value


In [ ]:
def get_pixel_magn_diff(img, center, x, y):
    new_value = 0
    try:
        new_value = abs(int(img[x][y]) - int(center))
    except:
        pass
    return new_value


In [ ]:

def lbp_calculated_pixel(img, x, y):
    '''
     64 | 128 |   1
    ----------------
     32 |   0 |   2
    ----------------
     16 |   8 |   4    
    '''    
    center = img[x][y]
    val_ar = []
    val_ar.append(get_pixel_sign(img, center, x-1, y+1))     # top_right
    val_ar.append(get_pixel_sign(img, center, x, y+1))       # right
    val_ar.append(get_pixel_sign(img, center, x+1, y+1))     # bottom_right
    val_ar.append(get_pixel_sign(img, center, x+1, y))       # bottom
    val_ar.append(get_pixel_sign(img, center, x+1, y-1))     # bottom_left
    val_ar.append(get_pixel_sign(img, center, x, y-1))       # left
    val_ar.append(get_pixel_sign(img, center, x-1, y-1))     # top_left
    val_ar.append(get_pixel_sign(img, center, x-1, y))       # top
    
    power_val = [1, 2, 4, 8, 16, 32, 64, 128]
    val = 0
    for i in range(len(val_ar)):
        val += val_ar[i] * power_val[i]
    return val    


In [ ]:

def clbp_sign_vec(img, x, y):
    '''
     64 | 128 |   1
    ----------------
     32 |   0 |   2
    ----------------
     16 |   8 |   4    
    '''    
    center = img[x][y]
    val_ar = []
    val_ar.append(get_pixel_sign(img, center, x-1, y+1))     # top_right
    val_ar.append(get_pixel_sign(img, center, x, y+1))       # right
    val_ar.append(get_pixel_sign(img, center, x+1, y+1))     # bottom_right
    val_ar.append(get_pixel_sign(img, center, x+1, y))       # bottom
    val_ar.append(get_pixel_sign(img, center, x+1, y-1))     # bottom_left
    val_ar.append(get_pixel_sign(img, center, x, y-1))       # left
    val_ar.append(get_pixel_sign(img, center, x-1, y-1))     # top_left
    val_ar.append(get_pixel_sign(img, center, x-1, y))       # top

    power_val = [1, 2, 4, 8, 16, 32, 64, 128]
    val = 0
    # print(val_ar)
    for i in range(len(val_ar)):
        val += val_ar[i] * power_val[i]
    return val     


In [ ]:

def clbp_Mag_vec(img, x, y):  
    center = img[x][y]
    # print("center = ",center)

    val_ar = []
    # print(img[x-1][y+1],"\n")
    val_ar.append(get_pixel_magn_diff(img, center, x-1, y+1))     # top_right
    # print("center = ",center)
    # print(img[x-1][y+1],"\n")
    
    val_ar.append(get_pixel_magn_diff(img, center, x, y+1))       # right
    # print("center = ",center)
    # print(img[x][y+1],"\n")
    
    val_ar.append(get_pixel_magn_diff(img, center, x+1, y+1))     # bottom_right
    # print("center = ",center)
    # print(img[x+1][y+1],"\n")
    
    val_ar.append(get_pixel_magn_diff(img, center, x+1, y))       # bottom
    # print("center = ",center)
    # print(img[x+1][y],"\n")
    
    val_ar.append(get_pixel_magn_diff(img, center, x+1, y-1))     # bottom_left
    # print("center = ",center)
    # print(img[x+1][y-1],"\n")
    
    val_ar.append(get_pixel_magn_diff(img, center, x, y-1))       # left
    # print("center = ",center)
    # print(img[x][y-1],"\n")
    
    val_ar.append(get_pixel_magn_diff(img, center, x-1, y-1))     # top_left
    # print("center = ",center)
    # print(img[x-1][y-1],"\n")
    
    val_ar.append(get_pixel_magn_diff(img, center, x-1, y))       # top
    # print("center = ",center)
    # print(img[x-1][y],"\n")

    return val_ar    


In [ ]:
def t_paper(x,c):
    if(x>=c):
        return 1
    return 0

In [ ]:
def clbp_S_Operator(img,x,y):
    return clbp_sign_vec(img,x,y)

In [ ]:
def clbp_M_Operator(mp,P):
    c_threshold = np.mean(mp)
    tp = np.where(mp >= c_threshold, 1, 0)
    pp2 = (np.array(list(range(0,P))))
    return np.dot(tp,pp2)

In [ ]:
def calcLocalDifferences(img,P,R):
    ld = np.zeros((img.shape[0],img.shape[1],P),dtype=np.float)

    for y in range(0, len(img)):
        for x in range(0, len(img[0])):
            center = img[y,x]
            #pixels = getNeighboringPixels(img,R,P,x,y)              
            pixels = getNeighboringPixelsPaperVersion(img,R,P,x,y)              
            ld[y,x,0:P] = pixels-center

    return ld

In [ ]:
def clbp_C_Operator(img):
    c = np.mean(img)
    return np.where(img >= c, 1, 0)


In [ ]:
def LDSMT(ld):
    sp = np.where(ld >= 0,1,-1)
    mp = np.absolute(ld)
    return sp, mp

In [ ]:
def CLBP_S_riu2(sp,P):
    sp = np.where(sp >= 0, 1, 0)
    #sp_total = np.sum(sp,2)
    #u_value = calcTransitions(sp,P)
    #return np.where(u_value <= 2, sp_total, P+1)
    pp2 = 2**(np.array(list(range(0,P))))
    indexes = np.dot(sp,pp2)    
    return indexes

In [ ]:
def CLBP_M_riu2(mp,P):
    c = np.mean(mp)
    tp = np.where(mp >= c, 1, 0)
    # pp2 = np.array([1]*P)
    # tp_total = np.dot(tp,pp2.T)
    # u_value = calcTransitions(tp,P)
    # return np.where(u_value <= 2, tp_total, P+1)
    pp2 = 2**(np.array(list(range(0,P))))
    indexes = np.dot(tp,pp2)    
    return indexes

In [ ]:
def genMappingTable(P):
    mapTable = np.zeros(2**P)
    numElms = P+2
    for i in range(0,2**P):
        msb = 128 & i
        lsb = 1 & i
        count = format(i, '#010b').count('01') + format(i, '#010b').count('10') + ((msb >> (P-1)) ^ lsb)

        if count <= 2:
            mapTable[i] = bin(i).count('1')
        else:
            mapTable[i] = P+1

    return mapTable

In [ ]:
def getNeighboringPixelsPaperVersion(img,R,P,x,y):
    pixels = []

    indexes = np.array(list(range(0,P)),dtype=np.float)
    dy = -R * np.sin(2 * np.pi * indexes / P)
    dx = R * np.cos(2 * np.pi * indexes / P)

    dy = np.where(abs(dy) < 5.0e-10, 0, dy)
    dx = np.where(abs(dx) < 5.0e-10, 0, dx)

    for point in range(0, P):
        r = y + dy[point]
        c = x + dx[point]

        fr = math.floor(r)
        fc = math.floor(c)

        cr = math.ceil(r)
        cc = math.ceil(c)
        
        rr = np.round(r)
        rc = np.round(c)

        if abs(c-rc) < 10e-7 and abs(r-rr) < 10e-7:
            pixels.append(get_pixel_else_0(img, int(r), int(c)))
        else:
            tr = r - fr
            tc = c - fc

            w1 = (1 - tc) * (1 - tr)
            w2 =      tc  * (1 - tr)
            w3 = (1 - tc) *      tr 
            w4 =      tc  *      tr 

            value = w1*get_pixel_else_0(img, fr, fc) + w2*get_pixel_else_0(img, fr, cc) + \
                    w3*get_pixel_else_0(img, cr, fc) + w4*get_pixel_else_0(img, cr, cc)

            pixels.append(value)

    return pixels


In [ ]:
def chiSquared(p,q):
    return np.sum((p-q)**2/(p+q+1e-6))

Import dataset

In [ ]:
aluminium_foil = glob.glob('/content/drive/MyDrive/PS I G3/Dataset/KTH-TIPS 2-a/KTH-TIPS2-a/aluminium_foil/sample_a/*.*')
brown_bread = glob.glob('/content/drive/MyDrive/PS I G3/Dataset/KTH-TIPS 2-a/KTH-TIPS2-a/brown_bread/sample_a/*.*')
corduroy = glob.glob('/content/drive/MyDrive/PS I G3/Dataset/KTH-TIPS 2-a/KTH-TIPS2-a/corduroy/sample_a/*.*')
cork = glob.glob('/content/drive/MyDrive/PS I G3/Dataset/KTH-TIPS 2-a/KTH-TIPS2-a/cork/sample_a/*.*')
cotton = glob.glob('/content/drive/MyDrive/PS I G3/Dataset/KTH-TIPS 2-a/KTH-TIPS2-a/cotton/sample_a/*.*')
cracker = glob.glob('/content/drive/MyDrive/PS I G3/Dataset/KTH-TIPS 2-a/KTH-TIPS2-a/cracker/sample_a/*.*')
lettuce_leaf = glob.glob('/content/drive/MyDrive/PS I G3/Dataset/KTH-TIPS 2-a/KTH-TIPS2-a/lettuce_leaf/sample_a/*.*')
linen = glob.glob('/content/drive/MyDrive/PS I G3/Dataset/KTH-TIPS 2-a/KTH-TIPS2-a/linen/sample_a/*.*')
white_bread = glob.glob('/content/drive/MyDrive/PS I G3/Dataset/KTH-TIPS 2-a/KTH-TIPS2-a/white_bread/sample_a/*.*')
wood = glob.glob('/content/drive/MyDrive/PS I G3/Dataset/KTH-TIPS 2-a/KTH-TIPS2-a/wood/sample_a/*.*')
wool = glob.glob('/content/drive/MyDrive/PS I G3/Dataset/KTH-TIPS 2-a/KTH-TIPS2-a/wool/sample_a/*.*')


# In[5]:


data = []
labels = []
j= 0
for i in aluminium_foil:
    if j>10:
        break
    image=tf.keras.preprocessing.image.load_img(i, color_mode="rgb", target_size= (200,200))
    image=np.array(image)
    data.append(image)
    labels.append(0)
    j +=1
for i in brown_bread:
    if j>20:
        break
    image=tf.keras.preprocessing.image.load_img(i, color_mode="rgb", target_size= (200,200))
    image=np.array(image)
    data.append(image)
    labels.append(1)
    j +=1
for i in corduroy:
    if j>30:
        break
    image=tf.keras.preprocessing.image.load_img(i, color_mode="rgb", target_size= (200,200))
    image=np.array(image)
    data.append(image)
    labels.append(2)
    j +=1
for i in cork:
    if j>40:
        break
    image=tf.keras.preprocessing.image.load_img(i, color_mode="rgb", target_size= (200,200))
    image=np.array(image)
    data.append(image)
    labels.append(3)
    j+=1
for i in cotton:
    if j>50:
        break
    image=tf.keras.preprocessing.image.load_img(i, color_mode="rgb", target_size= (200,200))
    image=np.array(image)
    data.append(image)
    labels.append(4)
    j+=1
for i in cracker:
    if j> 60:
        break
    image=tf.keras.preprocessing.image.load_img(i, color_mode="rgb", target_size= (200,200))
    image=np.array(image)
    data.append(image)
    labels.append(5)
    j+=1
for i in lettuce_leaf:
    if j> 70:
        break
    image=tf.keras.preprocessing.image.load_img(i, color_mode="rgb", target_size= (200,200))
    image=np.array(image)
    data.append(image)
    labels.append(6)
    j+=1
for i in linen:
    if j> 80:
        break
    image=tf.keras.preprocessing.image.load_img(i, color_mode="rgb", target_size= (200,200))
    image=np.array(image)
    data.append(image)
    labels.append(7)
    j+=1
for i in white_bread:
    if j> 90:
        break
    image=tf.keras.preprocessing.image.load_img(i, color_mode="rgb", target_size= (200,200))
    image=np.array(image)
    data.append(image)
    labels.append(8)
    j+=1
for i in wood:
    if j> 100:
        break
    image=tf.keras.preprocessing.image.load_img(i, color_mode="rgb", target_size= (200,200))
    image=np.array(image)
    data.append(image)
    labels.append(9)
    j+=1
for i in wool:
    if j> 110:
        break
    image=tf.keras.preprocessing.image.load_img(i, color_mode="rgb", target_size= (200,200))
    image=np.array(image)
    data.append(image)
    labels.append(10)
    j+=1

data = np.array(data)
labels = np.array(labels)

dataset for VISTEX

In [ ]:
dataset = glob.glob('/content/drive/MyDrive/PS I G3/Dataset/VisTex/VisTex/FLAT/128x128/*.*.*')
data = []
labels = []
for files in dataset:
    filename = os.path.basename(files)
    labelname = filename.split(".")[0]
    if labelname == 'Bark':
        image=tf.keras.preprocessing.image.load_img(files, color_mode="rgb", target_size= (200,200))
        image=np.array(image)
        data.append(image)
        labels.append(0)
    if labelname == 'Brick':
        image=tf.keras.preprocessing.image.load_img(files, color_mode="rgb", target_size= (200,200))
        image=np.array(image)
        data.append(image)
        labels.append(1)
    if labelname == 'Buildings':
        image=tf.keras.preprocessing.image.load_img(files, color_mode="rgb", target_size= (200,200))
        image=np.array(image)
        data.append(image)
        labels.append(2)
    if labelname == 'Clouds':
        image=tf.keras.preprocessing.image.load_img(files, color_mode="rgb", target_size= (200,200))
        image=np.array(image)
        data.append(image)
        labels.append(3)
    if labelname == 'Fabric':
        image=tf.keras.preprocessing.image.load_img(files, color_mode="rgb", target_size= (200,200))
        image=np.array(image)
        data.append(image)
        labels.append(4)
    if labelname == 'Flowers':
        image=tf.keras.preprocessing.image.load_img(files, color_mode="rgb", target_size= (200,200))
        image=np.array(image)
        data.append(image)
        labels.append(5)
    if labelname == 'Food':
        image=tf.keras.preprocessing.image.load_img(files, color_mode="rgb", target_size= (200,200))
        image=np.array(image)
        data.append(image)
        labels.append(6)
    if labelname == 'Grass':
        image=tf.keras.preprocessing.image.load_img(files, color_mode="rgb", target_size= (200,200))
        image=np.array(image)
        data.append(image)
        labels.append(7)
    if labelname == 'Leaves':
        image=tf.keras.preprocessing.image.load_img(files, color_mode="rgb", target_size= (200,200))
        image=np.array(image)
        data.append(image)
        labels.append(8)
    if labelname == 'Metal':
        image=tf.keras.preprocessing.image.load_img(files, color_mode="rgb", target_size= (200,200))
        image=np.array(image)
        data.append(image)
        labels.append(9)
    if labelname == 'Misc':
        image=tf.keras.preprocessing.image.load_img(files, color_mode="rgb", target_size= (200,200))
        image=np.array(image)
        data.append(image)
        labels.append(10)
    if labelname == 'Paintings':
        image=tf.keras.preprocessing.image.load_img(files, color_mode="rgb", target_size= (200,200))
        image=np.array(image)
        data.append(image)
        labels.append(11)
    if labelname == 'Sand':
        image=tf.keras.preprocessing.image.load_img(files, color_mode="rgb", target_size= (200,200))
        image=np.array(image)
        data.append(image)
        labels.append(12)
    if labelname == 'Stone':
        image=tf.keras.preprocessing.image.load_img(files, color_mode="rgb", target_size= (200,200))
        image=np.array(image)
        data.append(image)
        labels.append(13)
    if labelname == 'Terrain':
        image=tf.keras.preprocessing.image.load_img(files, color_mode="rgb", target_size= (200,200))
        image=np.array(image)
        data.append(image)
        labels.append(14)
    if labelname == 'Tile':
        image=tf.keras.preprocessing.image.load_img(files, color_mode="rgb", target_size= (200,200))
        image=np.array(image)
        data.append(image)
        labels.append(15)
    if labelname == 'Water':
        image=tf.keras.preprocessing.image.load_img(files, color_mode="rgb", target_size= (200,200))
        image=np.array(image)
        data.append(image)
        labels.append(16)
    if labelname == 'WheresWaldo':
        image=tf.keras.preprocessing.image.load_img(files, color_mode="rgb", target_size= (200,200))
        image=np.array(image)
        data.append(image)
        labels.append(17)
    if labelname == 'Wood':
        image=tf.keras.preprocessing.image.load_img(files, color_mode="rgb", target_size= (200,200))
        image=np.array(image)
        data.append(image)
        labels.append(18)

data = np.array(data)
labels = np.array(labels)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

CLBP Github

Train dataset histogram

In [ ]:
radius = 1
numPoints = 8

clbp_s = []
clbp_m = []
clbp_mc = []
clbp_s_mc = []
clbp_sm = []
clbp_smc = []

labels = []

result_y_train_clbp_s = []
result_y_train_clbp_m = []
result_y_train_clbp_mc = []
result_y_train_clbp_s_mc = []
result_y_train_clbp_sm = []
result_y_train_clbp_smc = []

nn_classifier_clbp_s = make_pipeline(StandardScaler(), SVC(gamma='auto'))
nn_classifier_clbp_m = make_pipeline(StandardScaler(), SVC(gamma='auto'))
nn_classifier_clbp_mc = make_pipeline(StandardScaler(), SVC(gamma='auto'))
nn_classifier_clbp_s_mc = make_pipeline(StandardScaler(), SVC(gamma='auto'))
nn_classifier_clbp_sm = make_pipeline(StandardScaler(), SVC(gamma='auto'))
nn_classifier_clbp_smc = make_pipeline(StandardScaler(), SVC(gamma='auto'))

mappingTable = genMappingTable(numPoints)

# train
i = 0
for imgdata in X_train:
    i+=1
    print(i)

    img_bgr = imgdata
    height, width, channel = img_bgr.shape
    img_gray = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2GRAY)
    img = img_gray
    # print("height,width",height,width)
    # cv2.imshow("img",img)
    # cv2.waitKey(0)
    # cv2.destroyAllWindows()
    # img = img/255
    # img = (img-np.mean(img))/np.std(img)*20+128

    dp = calcLocalDifferences(img,numPoints,radius)
    sp, mp = LDSMT(dp)

    # CLBP_C
    clbp_c = clbp_C_Operator(img)

    # CLBP_S_riu2
    clbp_s_riu2 = CLBP_S_riu2(sp,numPoints)
    mapped_clbp_s_riu2 = mappingTable[clbp_s_riu2]
    (hist_s, bins) = np.histogram(mapped_clbp_s_riu2.ravel(), bins=numPoints+2)
    clbp_s.append(hist_s.ravel())
    
    # CLBP_M_riu2
    clbp_m_riu2 = CLBP_M_riu2(mp,numPoints)
    mapped_clbp_m_riu2 = mappingTable[clbp_m_riu2]
    (hist_m, bins) = np.histogram(mapped_clbp_m_riu2.ravel(), bins=numPoints+2)
    clbp_m.append(hist_m.ravel())

    # CLBP_M/C
    hist_mc, xedges, yedges = np.histogram2d(mapped_clbp_m_riu2.ravel(), clbp_c.ravel(),bins=[numPoints+2,2])
    clbp_mc.append(hist_mc.ravel())

    # CLBP_S_M/C
    hist_s_mc = np.concatenate((hist_s,hist_mc.ravel()),axis=0)
    clbp_s_mc.append(hist_s_mc.ravel())

    # CLBP_S/M
    hist_sm, xedges, yedges = np.histogram2d(mapped_clbp_s_riu2.ravel(),mapped_clbp_m_riu2.ravel(),bins=[numPoints+2,numPoints+2])
    clbp_sm.append(hist_sm.ravel())

    # CLBP_S/M/C
    clbp_mc_sum = np.where(clbp_c > 0, clbp_m_riu2+numPoints+2, clbp_m_riu2)
    hist_smc, xedges, yedges = np.histogram2d(mapped_clbp_s_riu2.ravel(), clbp_mc_sum.ravel(),bins=[numPoints+2,2*(numPoints+2)])
    clbp_smc.append(hist_smc.ravel())


                




1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133


Fitting the dataset

In [ ]:
# fitting
print("Fitting the models")

# model = KNeighborsClassifier(n_neighbors = 1, weights='uniform', algorithm='auto', metric=chiSquared)
# model.fit(data,y_train)

nn_classifier_clbp_s.fit(clbp_s,y_train)
nn_classifier_clbp_m.fit(clbp_m,y_train)
nn_classifier_clbp_mc.fit(clbp_mc,y_train)
nn_classifier_clbp_s_mc.fit(clbp_s_mc,y_train)
nn_classifier_clbp_sm.fit(clbp_sm,y_train)
nn_classifier_clbp_smc.fit(clbp_smc,y_train)

Fitting the models


Pipeline(memory=None,
         steps=[('standardscaler',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('svc',
                 SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None,
                     coef0=0.0, decision_function_shape='ovr', degree=3,
                     gamma='auto', kernel='rbf', max_iter=-1, probability=False,
                     random_state=None, shrinking=True, tol=0.001,
                     verbose=False))],
         verbose=False)

Test dataset histogram


In [ ]:
result_labels_clbp_s=[]
result_labels_clbp_m=[]
result_labels_clbp_mc=[]
result_labels_clbp_s_mc=[]
result_labels_clbp_sm=[]
result_labels_clbp_smc=[]

In [ ]:
# test
print("Test steps:")
i = 0
for imgdata in X_test:
    print(i)
    # image_file = '/content/00000'+str(i)+'.bmp'
    img_bgr = imgdata
    height, width, channel = img_bgr.shape
    img_gray = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2GRAY)
    img = img_gray
    dp = calcLocalDifferences(img,numPoints,radius)
    sp, mp = LDSMT(dp)

    # CLBP_C
    clbp_c = clbp_C_Operator(img)

    # CLBP_S_riu2
    clbp_s_riu2 = CLBP_S_riu2(sp,numPoints)
    mapped_clbp_s_riu2 = mappingTable[clbp_s_riu2]
    (hist_s, bins) = np.histogram(mapped_clbp_s_riu2.ravel(), bins=numPoints+2)
    result_labels_clbp_s.append((y_test[i],nn_classifier_clbp_s.predict([hist_s.ravel()])))
    
    # CLBP_M_riu2
    clbp_m_riu2 = CLBP_M_riu2(mp,numPoints)
    mapped_clbp_m_riu2 = mappingTable[clbp_m_riu2]
    (hist_m, bins) = np.histogram(mapped_clbp_m_riu2.ravel(), bins=numPoints+2)
    result_labels_clbp_m.append((y_test[i],nn_classifier_clbp_m.predict([hist_m.ravel()])))


    # CLBP_M/C
    hist_mc, xedges, yedges = np.histogram2d(mapped_clbp_m_riu2.ravel(), clbp_c.ravel(),bins=[numPoints+2,2])
    result_labels_clbp_mc.append((y_test[i],nn_classifier_clbp_mc.predict([hist_mc.ravel()])))

    # CLBP_S_M/C
    hist_s_mc = np.concatenate((hist_s,hist_mc.ravel()),axis=0)
    result_labels_clbp_s_mc.append((y_test[i],nn_classifier_clbp_s_mc.predict([hist_s_mc.ravel()])))

    # CLBP_S/M
    hist_sm, xedges, yedges = np.histogram2d(mapped_clbp_s_riu2.ravel(),mapped_clbp_m_riu2.ravel(),bins=[numPoints+2,numPoints+2])
    result_labels_clbp_sm.append((y_test[i],nn_classifier_clbp_sm.predict([hist_sm.ravel()])))

    # CLBP_S/M/C
    clbp_mc_sum = np.where(clbp_c > 0, clbp_m_riu2+numPoints+2, clbp_m_riu2)
    hist_smc, xedges, yedges = np.histogram2d(mapped_clbp_s_riu2.ravel(), clbp_mc_sum.ravel(),bins=[numPoints+2,2*(numPoints+2)])
    result_labels_clbp_smc.append((y_test[i],nn_classifier_clbp_smc.predict([hist_smc.ravel()])))
    
    i+=1

Test steps:
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33


Only combined MS histogram

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=5)
neigh.fit(hist_clbp_ms, y_train)
cv2_imshow(X_test[4])
x_test_12 = cv2.cvtColor(X_test[4], cv2.COLOR_BGR2GRAY)
cv2_imshow(x_test_12)
print(neigh.predict([hist_clbp_ms_test[4]]))
acc_ms = neigh.score(hist_clbp_ms_test, y_test)
# print(neigh.summary())
print("[INFO] histogram accuracy: {:.2f}%".format(acc_ms * 100))

NameError: ignored

only S histogram

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=5)
neigh.fit(clbp_s, y_train)
cv2_imshow(X_test[4])
x_test_12 = cv2.cvtColor(X_test[4], cv2.COLOR_BGR2GRAY)
cv2_imshow(x_test_12)

print(neigh.predict([clbp_s_test[4]]))
acc_s = neigh.score(clbp_s_test, y_test)
# print(neigh.summary())
print("[INFO] histogram accuracy: {:.2f}%".format(acc_s * 100))

Predictions results for CLBP_S_riu2

In [ ]:
pred = 0
numTest = X_test.shape[0]
for j in range(numTest):
    pred += (int(result_labels_clbp_s[j][0] == result_labels_clbp_s[j][1][0]))
    
print('predictions =',pred)
print('accuracy =',pred/(numTest)*100)

predictions = 12
accuracy = 35.294117647058826


Predictions results for CLBP_M_riu2

In [ ]:
pred = 0
numTest = X_test.shape[0]
for j in range(numTest):
    pred += (int(result_labels_clbp_m[j][0] == result_labels_clbp_m[j][1][0]))
    
print('predictions =',pred)
print('accuracy =',pred/(numTest)*100)

predictions = 15
accuracy = 44.11764705882353


Predictions results for CLBP_M/C

In [ ]:
pred = 0
numTest = X_test.shape[0]
for j in range(numTest):
    pred += (int(result_labels_clbp_mc[j][0] == result_labels_clbp_mc[j][1][0]))
    
print('predictions =',pred)
print('accuracy =',pred/(numTest)*100)

predictions = 13
accuracy = 38.23529411764706


Predictions results for CLBP_S_M/C

In [ ]:
pred = 0
numTest = X_test.shape[0]
for j in range(numTest):
    pred += (int(result_labels_clbp_s_mc[j][0] == result_labels_clbp_s_mc[j][1][0]))
    
print('predictions =',pred)
print('accuracy =',pred/(numTest)*100)

predictions = 15
accuracy = 44.11764705882353


Predictions results for CLBP_S/M

In [ ]:
pred = 0
numTest = X_test.shape[0]
for j in range(numTest):
    pred += (int(result_labels_clbp_sm[j][0] == result_labels_clbp_smc[j][1][0]))
    
print('predictions =',pred)
print('accuracy =',pred/(numTest)*100)

predictions = 14
accuracy = 41.17647058823529


Predictions results for CLBP_S/M/C

In [ ]:
pred = 0
numTest = X_test.shape[0]
for j in range(numTest):
    pred += (int(result_labels_clbp_smc[j][0] == result_labels_clbp_smc[j][1][0]))
    
print('predictions =',pred)
print('accuracy =',pred/(numTest)*100)

predictions = 14
accuracy = 41.17647058823529


with 220

In [ ]:
pred = 0
numTest = X_test.shape[0]
for j in range(numTest):
    pred += (int(result_labels_clbp_smc[j][0] == result_labels_clbp_smc[j][1][0]))
    
print('predictions =',pred)
print('accuracy =',pred/(numTest)*100)

predictions = 40
accuracy = 88.88888888888889


Plotting decision boundaries of KNN classifier

SVC Classification

In [ ]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
clf1 = make_pipeline(StandardScaler(), SVC(gamma='auto'))
clf2 = make_pipeline(StandardScaler(), SVC(gamma='auto'))
clf3 = make_pipeline(StandardScaler(), SVC(gamma='auto'))
clf4 = make_pipeline(StandardScaler(), SVC(gamma='auto'))
clf5 = make_pipeline(StandardScaler(), SVC(gamma='auto'))
clf6 = make_pipeline(StandardScaler(), SVC(gamma='auto'))

clf1.fit(clbp_s,y_train)
clf2.fit(clbp_m,y_train)
clf3.fit(clbp_mc,y_train)
clf4.fit(clbp_s_mc,y_train)
clf5.fit(clbp_sm,y_train)
clf6.fit(clbp_smc,y_train)
# Pipeline(steps=[('standardscaler', StandardScaler()),('svc', SVC(gamma='auto'))])


Pipeline(memory=None,
         steps=[('standardscaler',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('svc',
                 SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None,
                     coef0=0.0, decision_function_shape='ovr', degree=3,
                     gamma='auto', kernel='rbf', max_iter=-1, probability=False,
                     random_state=None, shrinking=True, tol=0.001,
                     verbose=False))],
         verbose=False)

In [ ]:
result_labels_clbp_s2=[]
result_labels_clbp_m2=[]
result_labels_clbp_mc2=[]
result_labels_clbp_s_mc2=[]
result_labels_clbp_sm2=[]
result_labels_clbp_smc_2=[]

In [ ]:
# test
print("Test steps:")
i = 0
for imgdata in X_test:
    print(i)
    # image_file = '/content/00000'+str(i)+'.bmp'
    img_bgr = imgdata
    height, width, channel = img_bgr.shape
    img_gray = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2GRAY)
    img = img_gray
    dp = calcLocalDifferences(img,numPoints,radius)
    sp, mp = LDSMT(dp)

    # CLBP_C
    clbp_c = clbp_C_Operator(img)

    # CLBP_S_riu2
    clbp_s_riu2 = CLBP_S_riu2(sp,numPoints)
    mapped_clbp_s_riu2 = mappingTable[clbp_s_riu2]
    (hist_s, bins) = np.histogram(mapped_clbp_s_riu2.ravel(), bins=numPoints+2)
    result_labels_clbp_s2.append((y_test[i],clf1.predict([hist_s.ravel()])))
    
    # CLBP_M_riu2
    clbp_m_riu2 = CLBP_M_riu2(mp,numPoints)
    mapped_clbp_m_riu2 = mappingTable[clbp_m_riu2]
    (hist_m, bins) = np.histogram(mapped_clbp_m_riu2.ravel(), bins=numPoints+2)
    result_labels_clbp_m2.append((y_test[i],clf2.predict([hist_m.ravel()])))


    # CLBP_M/C
    hist_mc, xedges, yedges = np.histogram2d(mapped_clbp_m_riu2.ravel(), clbp_c.ravel(),bins=[numPoints+2,2])
    result_labels_clbp_mc2.append((y_test[i],clf3.predict([hist_mc.ravel()])))

    # CLBP_S_M/C
    hist_s_mc = np.concatenate((hist_s,hist_mc.ravel()),axis=0)
    result_labels_clbp_s_mc2.append((y_test[i],clf4.predict([hist_s_mc.ravel()])))

    # CLBP_S/M
    hist_sm, xedges, yedges = np.histogram2d(mapped_clbp_s_riu2.ravel(),mapped_clbp_m_riu2.ravel(),bins=[numPoints+2,numPoints+2])
    result_labels_clbp_sm2.append((y_test[i],clf5.predict([hist_sm.ravel()])))

    # CLBP_S/M/C
    clbp_mc_sum = np.where(clbp_c > 0, clbp_m_riu2+numPoints+2, clbp_m_riu2)
    hist_smc, xedges, yedges = np.histogram2d(mapped_clbp_s_riu2.ravel(), clbp_mc_sum.ravel(),bins=[numPoints+2,2*(numPoints+2)])
    result_labels_clbp_s_mc2.append((y_test[i],clf6.predict([hist_smc.ravel()])))
    
    i+=1

Test steps:
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220


SVC prediction

In [ ]:
pred = 0
numTest = X_test.shape[0]
for j in range(numTest):
    pred += (int(result_labels_clbp_smc[j][0] == result_labels_clbp_smc[j][1][0]))
    
print('predictions =',pred)
print('accuracy =',pred/(numTest)*100)

NameError: ignored

KNN WITH 1100 IMAGES


In [ ]:
pred = 0
numTest = X_test.shape[0]
for j in range(numTest):
    pred += (int(result_labels_clbp_smc[j][0] == result_labels_clbp_smc[j][1][0]))
    
print('predictions =',pred)
print('accuracy =',pred/(numTest)*100)

predictions = 219
accuracy = 99.09502262443439


s riu2


In [ ]:
pred = 0
numTest = X_test.shape[0]
for j in range(numTest):
    pred += (int(result_labels_clbp_s[j][0] == result_labels_clbp_s[j][1][0]))
    
print('predictions =',pred)
print('accuracy =',pred/(numTest)*100)

predictions = 204
accuracy = 92.3076923076923


m

In [ ]:
pred = 0
numTest = X_test.shape[0]
for j in range(numTest):
    pred += (int(result_labels_clbp_m[j][0] == result_labels_clbp_m[j][1][0]))
    
print('predictions =',pred)
print('accuracy =',pred/(numTest)*100)

predictions = 201
accuracy = 90.9502262443439


m_c

In [ ]:
pred = 0
numTest = X_test.shape[0]
for j in range(numTest):
    pred += (int(result_labels_clbp_mc[j][0] == result_labels_clbp_mc[j][1][0]))
    
print('predictions =',pred)
print('accuracy =',pred/(numTest)*100)

predictions = 213
accuracy = 96.38009049773756


s_m/c

In [ ]:
pred = 0
numTest = X_test.shape[0]
for j in range(numTest):
    pred += (int(result_labels_clbp_s_mc[j][0] == result_labels_clbp_s_mc[j][1][0]))
    
print('predictions =',pred)
print('accuracy =',pred/(numTest)*100)

predictions = 216
accuracy = 97.73755656108597


s/m

In [ ]:
pred = 0
numTest = X_test.shape[0]
for j in range(numTest):
    pred += (int(result_labels_clbp_sm[j][0] == result_labels_clbp_smc[j][1][0]))
    
print('predictions =',pred)
print('accuracy =',pred/(numTest)*100)

predictions = 212
accuracy = 95.92760180995475


SVC WITH 1100 IMAGES

s/m/c

In [ ]:
pred = 0
numTest = X_test.shape[0]
for j in range(numTest):
    pred += (int(result_labels_clbp_smc[j][0] == result_labels_clbp_smc[j][1][0]))
    
print('predictions =',pred)
print('accuracy =',pred/(numTest)*100)

predictions = 212
accuracy = 95.92760180995475


clbp s

In [ ]:
pred = 0
numTest = X_test.shape[0]
for j in range(numTest):
    pred += (int(result_labels_clbp_s2[j][0] == result_labels_clbp_s2[j][1][0]))
    
print('predictions =',pred)
print('accuracy =',pred/(numTest)*100)

predictions = 185
accuracy = 83.710407239819


clbp m

In [ ]:
pred = 0
numTest = X_test.shape[0]
for j in range(numTest):
    pred += (int(result_labels_clbp_m2[j][0] == result_labels_clbp_m2[j][1][0]))
    
print('predictions =',pred)
print('accuracy =',pred/(numTest)*100)

predictions = 193
accuracy = 87.33031674208145


clbp m_c

In [ ]:
pred = 0
numTest = X_test.shape[0]
for j in range(numTest):
    pred += (int(result_labels_clbp_mc2[j][0] == result_labels_clbp_mc2[j][1][0]))
    
print('predictions =',pred)
print('accuracy =',pred/(numTest)*100)

predictions = 212
accuracy = 95.92760180995475


clbp s_m/c

In [ ]:
pred = 0
numTest = X_test.shape[0]
for j in range(numTest):
    pred += (int(result_labels_clbp_s_mc2[j][0] == result_labels_clbp_s_mc2[j][1][0]))
    
print('predictions =',pred)
print('accuracy =',pred/(numTest)*100)

predictions = 215
accuracy = 97.28506787330316


clbp sm

In [ ]:
pred = 0
numTest = X_test.shape[0]
for j in range(numTest):
    pred += (int(result_labels_clbp_sm2[j][0] == result_labels_clbp_sm2[j][1][0]))
    
print('predictions =',pred)
print('accuracy =',pred/(numTest)*100)

predictions = 212
accuracy = 95.92760180995475


clbp smc